https://github.com/ShopRunner/jupyter-notify
```
pip install jupyternotify
```

In [39]:
import tensorflow as tf
import numpy as np
import ast
import os
import time
import datetime
import data_helpers
from tensorflow.contrib import learn
import pickle
import pandas as pd
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [36]:
class TextCNN:

    def __init__(
      self, sequence_length, num_classes, vocab_size, embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):


        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(self.embedded_chars_expanded,W,strides=[1, 1, 1, 1],padding="VALID", name="conv")
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                pooled = tf.nn.max_pool(h, ksize=[1, sequence_length - filter_size + 1, 1, 1],strides=[1, 1, 1, 1],padding='VALID',name="pool")
                pooled_outputs.append(pooled)


        num_filters_total = num_filters * len(filter_sizes)
        print(num_filters_total)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        

        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
                            # add@@@@@@@@
            features_size = 5
            self.features = tf.Variable(tf.truncated_normal([num_filters_total, features_size], stddev=0.1))
            # 384 * 5
            self.f_h = tf.matmul(self.h_drop, self.features, )
              # (None * 383) *(384 * 5) 
            print(self.h_drop) #None * 384 
            print(self.f_h) # Nonw * 5



        with tf.name_scope("output"):
            W = tf.get_variable("W", shape=[features_size, num_classes], initializer=tf.contrib.layers.xavier_initializer())
            #W = tf.get_variable("W", shape=[num_filters_total, num_classes], initializer=tf.contrib.layers.xavier_initializer())
            

            
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            #self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.scores = tf.nn.xw_plus_b(self.f_h, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [33]:
# Parameters
# ==================================================

# Data loading params
dev_sample_percentage = 0.001
positive_data_file = "data/amazon/book_pos.txt"#"./data/rt-polarity.pos"
negative_data_file = "data/amazon/book_neg.txt"#"./data/rt-polarity.neg" #"Data source for the negative data.")

# Model Hyperparameters
embedding_dim = 128 #, "Dimensionality of character embedding (default: 128)")
filter_sizes = "3,4,5"#, "Comma-separated filter sizes (default: '3,4,5')")
num_filters=128#, "Number of filters per filter size (default: 128)")
dropout_keep_prob=0.5#, "Dropout keep probability (default: 0.5)")
l2_reg_lambda=0.0#, "L2 regularization lambda (default: 0.0)")

# Training parameters
batch_size=64#, "Batch Size (default: 64)")
num_epochs=200#, "Number of training epochs (default: 200)")
evaluate_every=100#, "Evaluate model on dev set after this many steps (default: 100)")
num_checkpoints=5#, "Number of checkpoints to store (default: 5)")
# Misc Parameters
allow_soft_placement=True#, "Allow device soft device placement")
log_device_placement=False#, "Log placement of ops on devices")

#FLAGS = tf.flags.FLAGS
#FLAGS._parse_flags()
#print("\nParameters:")
#for attr, value in sorted(FLAGS.__flags.items()):
#    print("{}={}".format(attr.upper(), value))
#print("")

In [34]:
%%notify
x_text, y = data_helpers.load_data_and_labels(positive_data_file, negative_data_file)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [20]:
x_text[0]

'love the characters love the location ! i cannot wait for the next book in the series i will certainly read it'

In [21]:
%%notify
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

<IPython.core.display.Javascript object>

In [22]:
%%notify
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

dev_sample_index = -1 * int(dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

del x, y, x_shuffled, y_shuffled

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 203998
Train/Dev split: 999000/1000


<IPython.core.display.Javascript object>

In [24]:
chunk_size =  int(len(x_train)/10)
print(chunk_size)
for i in range(0, len(x_train), chunck_size):
    print(i)
    end = i+chunck_size
    if end < len(x_train):
        chunk = x_train[i:  end]
    else:
        chunk = x_train[i:]
        end = len(x_train)
    with open("data/x_train_{}.pkl".format(end), "wb") as f:
        pickle.dump(chunk, f, protocol = 4)

99900
0


OSError: [Errno 22] Invalid argument

In [15]:
all_size = 999000
chunk_size = 99900
x_train =[]
for i in range(0, all_size, chunck_size):
    print(i)
    end = i+chunk_size
    if end > len(reviews):
        end = all_size
    with open("data/x_train_{}.pkl".format(end), "rb") as f:
        x_train += pickle.load(f)

0
99900
199800
299700
399600
499500
599400
699300
799200
899100


In [ ]:
with open("data/y_train.pkl", "wb") as f:
        pickle.dump(y_train, f, protocol = 4)
with open("data/x_dev.pkl", "wb") as f:
        pickle.dump(x_dev, f, protocol = 4)
with open("data/y_dev.pkl", "wb") as f:
        pickle.dump(x_dev, f, protocol = 4)

In [37]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
tf.reset_default_graph()
with sess.as_default():
        
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=list(map(int, filter_sizes.split(","))),
            num_filters=num_filters,
            l2_reg_lambda=l2_reg_lambda)

384
Tensor("dropout/dropout/mul:0", shape=(?, 384), dtype=float32)
Tensor("dropout/MatMul:0", shape=(?, 5), dtype=float32)


In [ ]:
# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=list(map(int, filter_sizes.split(","))),
            num_filters=num_filters,
            l2_reg_lambda=l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)


        batches = data_helpers.batch_iter(list(zip(x_train, y_train)), batch_size, num_epochs)
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            #if current_step % checkpoint_every == 0:
            #    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
            #    print("Saved model checkpoint to {}\n".format(path))

384
Tensor("dropout/dropout/mul:0", shape=(?, 384), dtype=float32)
Tensor("dropout/MatMul:0", shape=(?, 5), dtype=float32)
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; u

2018-04-13T17:46:54.503401: step 93, loss 1.54335, acc 0.546875
2018-04-13T17:47:05.170367: step 94, loss 0.864733, acc 0.671875
2018-04-13T17:47:15.346336: step 95, loss 1.92825, acc 0.46875
2018-04-13T17:47:26.632422: step 96, loss 1.21186, acc 0.640625
2018-04-13T17:47:37.104371: step 97, loss 1.16195, acc 0.703125
2018-04-13T17:47:47.626065: step 98, loss 1.5543, acc 0.59375
2018-04-13T17:47:58.326876: step 99, loss 1.90125, acc 0.53125
2018-04-13T17:48:09.051032: step 100, loss 1.35579, acc 0.625

Evaluation:
2018-04-13T17:56:46.564649: step 100, loss 0.451258, acc 0.774

2018-04-13T17:56:57.670711: step 101, loss 1.51227, acc 0.578125
2018-04-13T17:57:08.404039: step 102, loss 1.59222, acc 0.625
2018-04-13T17:57:19.020346: step 103, loss 1.56445, acc 0.53125
2018-04-13T17:57:29.867039: step 104, loss 0.780058, acc 0.78125
2018-04-13T17:57:40.757858: step 105, loss 1.67472, acc 0.53125
2018-04-13T17:57:52.043071: step 106, loss 0.911877, acc 0.65625
2018-04-13T17:58:02.591407: ste